In [ ]:
import math
import seaborn as sns
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
class Neuron:
    def __init__(self, value, _children=(), _op='', label=''):
        self.value = value
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children);
        self._op = _op;
        self.label = label # for visualization

    def __repr__(self):
        stringVal = f"{self.value}";
        return stringVal

    def __add__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value + other.value, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value * other.value, (self, other), '*')
        def _backward():
            self.grad += other.value * out.grad
            other.grad += self.value * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Neuron(self.value**other, (self, ), f'**{other}')
        def _backward():
            self.grad += other * (self.value ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        return self * other**-1

    def exp(self, pos=1):
        x = self.value*pos
        out = Neuron(math.exp(x), (self, ), 'exp')
        def _backward():
            self.grad += out.value * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other): # other + self
        return self + other

    def __rsub__(self, other): # other - self
        return -(self - other)

    def __rmul__(self, other): # other * self
        return self * other

    def __rtruediv__(self, other): # other / self
        return self / other**-1

    def backward(self):
        topology = []
        visited = set()
        def build_topology(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topology(child)
                topology.append(v)
        build_topology(self)

        self.grad = 1.0
        for node in reversed(topology):
            node._backward()

    # Activation function
    def sigmoid(self):
        x  = self.value
        s = 1/(1+math.exp(-x))
        out = Neuron(s, (self, ), "sigmoid")

        def _backward():
            self.grad += (s*(1-s)) * out.grad
        out._backward = _backward

        return out

    def linear(self):
        out = Neuron(self.value, (self, ), "linear")

        def _backward():
            self.grad += 1*out.grad
        out._backward = _backward

        return out
    
    def relu(self):
        x = self.value
        r = x if x > 0 else 0
        dr = 1 if x > 0 else 0
        out = Neuron(r, (self, ), "relu")

        def _backward():
            self.grad += dr*out.grad
        out._backward = _backward

        return out
    
    def tanh(self):
        x = self.value
        t = (math.tanh(x))
        out = Neuron(t, (self, ), "tanh")

        def _backward():
            self.grad += (1-t**2) * out.grad
        out._backward = _backward

        return out
    
    def softmax(self, sum):
        return self.exp()/sum
    
    def swish(self):
        x = self.value
        sigmoid = 1/(1+math.exp(-x))
        swish = x*sigmoid
        out = Neuron(swish, (self, ), "swish")

        def _backward():
            self.grad += (sigmoid + x*sigmoid*(1-sigmoid)) * out.grad
        out._backward = _backward

        return out
    
    def leakyrelu(self, alpha=0.01):
        x = self.value
        lr = x if x > 0 else alpha*x
        dlr = 1 if x > 0 else alpha
        out = Neuron(lr, (self, ), "leakyrelu")

        def _backward():
            self.grad +=  dlr*out.grad
        out._backward = _backward

        return out

    def mse(self, target, n):
        x = self.value
        err = (target-x)**2/n
        out = Neuron(err, (self, ), "mse")

        def _backward():
            self.grad += -2*(target-x)/n * out.grad
        out._backward = _backward

        return out
    
    def binaryCrossEntropy(self, target, n):
        x = self.value
        err = -1/n*(target*math.log(x) + (1-target)*math.log(1-x))
        out = Neuron(err, (self, ), "bce")

        def _backward():
            self.grad += -1/n*(x-target)/(x*(1-x)) * out.grad
        out._backward = _backward

        return out

    def categoricalCrossEntropy(self, target, n):
        # Categorical Cross Entropy target will be an array cause the target will be encoded
        # For example if we have 3 classes 0, 1, 2 and the target for this instance is 1, then the target will be [0, 1, 0]
        # The predict or the output we get must be probability for every classes in target

        x = self.value
        c = len(target)
        sumOut = 0
        sumGrad = 0
        for i in range(c):
            sumOut += target[i]*math.log(x[i])
            sumGrad += target[i]/x[i]
        sumOut *= -1/n
        sumGrad *= -1/n

        out = Neuron(sum, (self, ), "cce")

        def _backward():
            self.grad += sumGrad * out.grad
        out._backward = _backward

        return out

    def update_weight(self, learning_rate):
        self.value = self.value-(learning_rate*self.grad)
        self.grad = 0.0

    def __float__(self):
        return float(self.value)
    
    def getGrad(self):
        return float(self.grad)

    def hello(self):
        print("hello")

In [ ]:
class FFNN19:
    def __init__(self,
                 hidden_layer_count=1,
                 input_neuron_count=1,
                 output_neuron_count=1,
                 activation_function="linear",
                 epoch=1,
                 loss="mse",
                 learning_rate=0.01
                 ):

        self.hidden_layer_count             = hidden_layer_count;
        self.input_neuron_count             = input_neuron_count;
        self.input_layer_neuron             = self.neuronGenerator(np.array,object=[0 for i in range(self.input_neuron_count)])
        self.output_neuron_count            = output_neuron_count;
        self._weight_layer_count            = hidden_layer_count+1;
        self.activation_function            = activation_function
        self.error_layer                    = []
        self.final_error                    = 0
        self.epoch                          = epoch
        self.loss                           = loss
        self.learning_rate                  = learning_rate
        self.weight_layers                  = [Neuron(0) for _ in range(self._weight_layer_count)];
        self.hidden_layers                  = [Neuron(0) for _ in range(hidden_layer_count)];
        self._activated_hidden_layers       = [Neuron(0) for _ in range(hidden_layer_count)];
        self._output_layer                  = [Neuron(0) for _ in range(output_neuron_count)];
        self._neuron_layer_set              = False;
        self._weight_layers_initialized     = False;
        self.outputs                        = []

    # Vectorize is not the most efficient solution (gw mager)
    def neuronGenerator(self,generator, *args, **kwargs):
        value = generator(*args, **kwargs)
        return np.vectorize(Neuron)(value)

    def setHiddenLayerNeuronCount(self,*args):
        if len(args)==0:
            for i in range(self.hidden_layer_count):
                self.hidden_layers[i] = self.neuronGenerator(np.zeros,(input())+1)
                self.hidden_layers[i][0] = Neuron(1)
            self._neuron_layer_set    = True
        elif len(args)==1:
            if len(args[0])!=self.hidden_layer_count:
                raise ValueError(f"Layer count mismatch: expected {self.hidden_layer_count} hidden layer, but {len(args[0])} were given")
            else:
                for i in range(self.hidden_layer_count):
                    self.hidden_layers[i] = self.neuronGenerator(np.zeros,args[0][i]+1)
                    self.hidden_layers[i][0] = Neuron(1)
                self._neuron_layer_set = True
        else:
            raise TypeError(f"FFNN19.setHiddenLayerNeuronCount() takes exactly 1 positional arguments but {len(args)} were given")

    def zeroWeightInitialization(self):
        if self._neuron_layer_set:
            for i in range(self._weight_layer_count):
                if i==0:
                    self.weight_layers[i] = self.neuronGenerator(np.zeros,(self.input_neuron_count+1,len(self.hidden_layers[i])))
                elif i==self._weight_layer_count-1:
                    self.weight_layers[i] = self.neuronGenerator(np.zeros,(len(self.hidden_layers[i-1]),self.output_neuron_count+1))
                else:
                    self.weight_layers[i] = self.neuronGenerator(np.zeros,(len(self.hidden_layers[i-1]),len(self.hidden_layers[i])))
            self._weight_layers_initialized     = True;
        else:
            raise ValueError(self._neuron_layer_set_message)

    def uniformWeightDistribution(self,lower,upper,seed):
        if self._neuron_layer_set:
            rng = np.random.default_rng(seed)
            for i in range(self._weight_layer_count):
                if i==0:
                    temp = self.neuronGenerator(rng.uniform,lower,upper,size=(self.input_neuron_count+1,len(self.hidden_layers[i])-1))
                elif i==self._weight_layer_count-1:
                    temp = self.neuronGenerator(rng.uniform,lower,upper,size=(len(self.hidden_layers[i-1]),self.output_neuron_count))
                else:
                    temp = self.neuronGenerator(rng.uniform,lower,upper,size=(len(self.hidden_layers[i-1]),len(self.hidden_layers[i])-1))
                tempBias = np.full((len(temp),1), Neuron(0), dtype=object)
                self.weight_layers[i] = np.hstack((tempBias,temp))
            self._weight_layers_initialized     = True;
        else:
            raise ValueError(f"Neuron layer count not set, please set neuron layer count before weight initialization")

    def normalWeightDistribution(self,mean,variance,seed):
        if self._neuron_layer_set:
            rng = np.random.default_rng(seed)
            for i in range(self._weight_layer_count):
                if i==0:
                    temp = self.neuronGenerator(rng.normal,mean,variance,size=(self.input_neuron_count+1,len(self.hidden_layers[i])-1))
                elif i==self._weight_layer_count-1:
                    temp = self.neuronGenerator(rng.normal,mean,variance,size=(len(self.hidden_layers[i-1]),self.output_neuron_count))
                else:
                    temp = self.neuronGenerator(rng.normal,mean,variance,size=(len(self.hidden_layers[i-1]),len(self.hidden_layers[i])-1))
                tempBias = np.full((len(temp),1), Neuron(0), dtype=object)
                self.weight_layers[i] = np.hstack((tempBias,temp))
            self._weight_layers_initialized     = True;
        else:
            raise ValueError(f"Neuron layer count not set, please set neuron layer count before weight initialization")

    # Single pass, masih bingung gmn caranya buat multiple
    def _feedforward(self,input_batch):
        if self._weight_layers_initialized:
            for input_data in input_batch:
                input_data = np.insert(input_data, 0, 1)
                # input_data.insert(0,1)
                self.input_layer_neuron = self.neuronGenerator(np.array,object=input_data)
                for i in range(self._weight_layer_count):
                    if i==0:
                        self.hidden_layers[i] = np.sum(self.input_layer_neuron*self.weight_layers[i].T,axis=1)
                        self.hidden_layers[i] = np.vectorize(lambda n: getattr(n, self.activation_function)())(self.hidden_layers[i])
                        # print(type(self.hidden_layers[i][1]))
                        self.hidden_layers[i][0] = Neuron(1)
                    elif i==self._weight_layer_count-1:
                        self._output_layer = np.sum(self.hidden_layers[i-1]*self.weight_layers[i].T,axis=1)
                        self._output_layer = np.vectorize(lambda n: getattr(n, self.activation_function)())(self._output_layer)
                    else:
                        self.hidden_layers[i] = np.sum(self.hidden_layers[i-1]*self.weight_layers[i].T,axis=1)
                        self.hidden_layers[i] = np.vectorize(lambda n: getattr(n, self.activation_function)())(self.hidden_layers[i])
                        self.hidden_layers[i][0] = Neuron(1)
                self.outputs.append(self._output_layer)
        else:
            raise ValueError(f"Weight layers not initialized, please initialize weight layers before feedforward")
    
    def showWeightDist(self,indices):
        for i in indices:
            sns.displot(self.weight_layers[i][:,1:].astype(float))

    def showGradDist(self,indices):
        vecfunc = np.vectorize(Neuron.getGrad,otypes=[float])
        for i in indices:
            sns.displot(vecfunc(self.weight_layers[i][:,1:]))

    # TODO, implementasi yg ini
    def _backpropagation(self, train_target):
        getattr(self, self.loss)(train_target)
        self.final_error.backward()
        # getattr(self, self.loss)(train_target)
        # print("niga",len(self.weight_layers))
        # print("maniga",len(self.weight_layers[0]))
        # print("black man",len(self.weight_layers[0][0]))
        # self.weight_layers = np.vectorize(lambda n: n.update_weight(self.learning_rate))(self.weight_layers)
        for i in self.weight_layers:
            for j in i:
                for k in j:
                    # print(k)
                    # print("grad:", k.grad)
                    k.update_weight(self.learning_rate)
        # for i in range(len(self.weight_layers)):
        #     for j in range(len(self.weight_layers[i])):
        #         for k in range(len(self.weight_layers[i][j])):
        #             # print(k)
        #             # k = k.update_weight(self.learning_rate)
        #             self.weight_layers[i][j][k].value = self.weight_layers[i][j][k].value-(self.learning_rate*self.weight_layers[i][j][k].grad)
        print("backprop")
        # raise NotImplementedError("Belum diimplement")

    def fit(self, train_data, train_target):
        for i in range(self.epoch):
            print("=============================epoch", i)
            self._feedforward(train_data)
            
            self._backpropagation(train_target)
            # print(self._output_layer)
            # print("trtg", train_target)
            # getattr(self, self.loss)(train_target)
        # raise NotImplementedError("Belum diimplement")

    def predict(self,data):
        self._feedforward(data)
        return self.outputs
        # raise NotImplementedError("Belum diimplement")
    
    def mse(self, target):
        n = len(target)
        for i in range(n):
            self.error_layer.append(self.outputs[i][1].mse(target[i], n))

        self.final_error = np.sum(self.error_layer)
    
    def binaryCrossEntropy(self, target):
        n = len(target)
        for i in range(n):
            self.error_layer.append(self.outputs.binaryCrossEntropy(target[i], n))
        
        self.final_error = np.sum(self.error_layer)
        # return -np.sum(target*np.log(output) + (np.subtract(1,target)*np.log(np.subtract(1, output))))/len(output)

    def categoricalCrossEntropy(self, target):
        n = len(target)
        for i in range(n):
            self.error_layer.append(self.outputs.categoricalCrossEntropy(target[i], n))

        self.final_error = np.sum(self.error_layer)

    def visualize(self):
        visual = nx.DiGraph()
        for i in range(self.input_neuron_count + 1):
            for j in range(1,len(self.weight_layers[0][i])):
                    tempW=float(self.weight_layers[0][i][j])
                    tempG=float(self.weight_layers[0][i][j].grad)

                    if i==0:
                        visual.add_edge(
                            f"Input Bias",
                            self.hidden_layers[0][j],
                            weight= f"{tempW:.2f}",
                            grad=f"{tempG:.2f}",
                        )
                    else:
                        visual.add_edge(
                            self.input_layer_neuron[i],
                            self.hidden_layers[0][j],
                            weight= f"{tempW:.2f}",
                            grad=f"{tempG:.2f}",
                        )

        for i in range(self.hidden_layer_count - 1): 
            for j in range(len(self.hidden_layers[i])):
                for k in range(1, len(self.hidden_layers[i + 1])):
                    tempW = float(self.weight_layers[i+1][j][k])
                    tempG = float(self.weight_layers[i+1][j][k].grad)
                    if j==0:
                        visual.add_edge(
                            f"#Bias {i}", 
                            self.hidden_layers[i + 1][k],
                            weight= f"{tempW:.2f}",
                            grad=f"{tempG:.2f}",
                        )
                        
                    else:
                        visual.add_edge(
                            self.hidden_layers[i][j], 
                            self.hidden_layers[i + 1][k],
                            weight= f"{tempW:.2f}",
                            grad=f"{tempG:.2f}",
                        )

        for i in range(len(self.hidden_layers[-1])):
            for j in range(1,len(self._output_layer)):
                tempW = float(self.weight_layers[-1][i][j])
                tempG = float(self.weight_layers[-1][i][j].grad)
                if i==0:
                    visual.add_edge(
                        f"Output Bias",
                        self._output_layer[j],
                        weight= f"{tempW:.2f}",
                        grad=f"{tempG:.2f}",
                    )

                else:
                    visual.add_edge(
                        self.hidden_layers[-1][i],
                        self._output_layer[j],
                        weight= f"{tempW:.2f}",
                        grad=f"{tempG:.2f}",
                    )

        pos = nx.nx_agraph.graphviz_layout(visual, prog="dot", args="-Grankdir=LR")
        nx.draw(
            visual, pos, with_labels=True, 
            node_color="lightblue", edge_color="gray", 
            node_size=2000, font_size=10
        )
        edge_weights = nx.get_edge_attributes(visual, 'weight')
        edge_grad = nx.get_edge_attributes(visual, 'grad')
        nx.draw_networkx_edge_labels(
            visual, pos,
            edge_labels=edge_weights,
            font_color='red',
            font_size=8,
            label_pos=0.7
        )

        nx.draw_networkx_edge_labels(
            visual, pos,
            edge_labels=edge_grad,
            font_color='blue',
            font_size=8,
            label_pos=0.3
        )
        plt.show()


In [ ]:
# Test dataset
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=10, test_size=10
)

model = FFNN19(
    input_neuron_count=784,
    hidden_layer_count=3,
    output_neuron_count=1,
    activation_function="sigmoid",
    epoch=5,
    loss="mse"
)

y_train = y_train.astype(int)
# print(len(X_train))
model.setHiddenLayerNeuronCount([4,5,8])
model.zeroWeightInitialization()
# model.uniformWeightDistribution(1,4,9)


model.fit(X_train, y_train)
y_pred = model.predict(X_test[0])
print(y_pred)


In [ ]:
print(type(model.weight_layers[0][1][1]))
print(model.weight_layers)
print(model.weight_layers[0])
print(model.weight_layers[0][1])
print(model.weight_layers[0][1][1])

In [ ]:
# model.visualize()

In [ ]:
modelInstance = FFNN19(
    input_neuron_count=2,
    hidden_layer_count=3,
    output_neuron_count=2,
    activation_function="sigmoid"
    )

In [ ]:
# modelInstance.setHiddenLayerNeuronCount([4,5,8])

In [ ]:
# modelInstance.uniformWeightDistribution(1,4,9)

In [ ]:
# modelInstance.setHiddenLayerNeuronCount([4,5,8])
# modelInstance.uniformWeightDistribution(1,4,9)
# modelInstance._feedforward([[5,8]])

In [ ]:
modelInstance._feedforward([[5,8]])
print(1,5,7,"\n")

print(modelInstance.weight_layers[0],"\n")
print(modelInstance.hidden_layers[0],"\n")
print(type(modelInstance.hidden_layers[0][1]),"\n")
print(modelInstance.weight_layers[1],"\n")
print(modelInstance.hidden_layers[1],"\n")
print(modelInstance.weight_layers[2],"\n")
print(modelInstance.hidden_layers[2],"\n")
print(modelInstance.weight_layers[3],"\n")
print(modelInstance._output_layer)
modelInstance.binaryCrossEntropy([1, 2, 4])
print("error layer ini")
print(modelInstance.error_layer)
modelInstance.final_error.backward()
# print(modelInstance.binaryCrossEntropy(modelInstance._output_layer, [1, 2]))

# print(modelInstance.weight_layers)

In [ ]:
# # print(modelInstance.outputs[1][0]._prev)
# modelInstance.visualize()


In [ ]:
# from graphviz import Digraph

# def trace(root):
#   # builds a set of all nodes and edges in a graph
#   nodes, edges = set(), set()
#   def build(v):
#     if v not in nodes:
#       nodes.add(v)
#       for child in v._prev:
#         edges.add((child, v))
#         build(child)
#   build(root)
#   return nodes, edges

# def draw_dot(root):
#   dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
#   nodes, edges = trace(root)
#   for n in nodes:
#     uid = str(id(n))
#     # for any value in the graph, create a rectangular ('record') node for it
#     dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.value, n.grad), shape='record')
#     if n._op:
#       # if this value is a result of some operation, create an op node for it
#       dot.node(name = uid + n._op, label = n._op)
#       # and connect this node to it
#       dot.edge(uid + n._op, uid)

#   for n1, n2 in edges:
#     # connect n1 to the op node of n2
#     dot.edge(str(id(n1)), str(id(n2)) + n2._op)

#   return dot

In [ ]:
# draw_dot(modelInstance.final_error)

In [ ]:


# n_input_neuron = 3 # jumlah neuron input
# n_input_instance = 1 # jumlah instance input
# n_hidden_layer = 2  # jumlah layer
# n_output_neuron = 1
# weight_layer_count = n_hidden_layer + 1


# input_layer_neuron = valueGenerator(np.random.randint,1,5,size=(n_input_instance,n_input_neuron))
# output_layer_neuron = valueGenerator(np.zeros,n_output_neuron)
# hidden_layer_neuron = [Value(0) for i in range(n_hidden_layer)]
# self.weight_layers = [Value(0) for i in range(weight_layer_count)]

# for i in range(n_hidden_layer):
#     hidden_layer_neuron[i] = np.zeros(int(input()))

# for i in range(weight_layer_count):
#     if i==0:
#         # weight_layer_neuron[i] = np.random.randint(5,size=(n_input_neuron,len(hidden_layer_neuron[i])))
#         self.weight_layers[i] = valueGenerator(np.random.randint,1,6,size=(n_input_neuron,len(hidden_layer_neuron[i])))
#     elif i==weight_layer_count-1:
#         self.weight_layers[i] = valueGenerator(np.random.randint,1,6,size=(len(hidden_layer_neuron[i-1]),n_output_neuron))
#         # weight_layer_neuron[i] = np.random.randint(5,size=(len(hidden_layer_neuron[i-1]),n_output_neuron))
#     else:
#         self.weight_layers[i] = valueGenerator(np.random.randint,1,6,size=(len(hidden_layer_neuron[i-1]),len(hidden_layer_neuron[i])))
#         # weight_layer_neuron[i] = np.random.randint(5,size=(len(hidden_layer_neuron[i-1]),len(hidden_layer_neuron[i])))

# for i in range(weight_layer_count):
#     if i==0:
#         hidden_layer_neuron[i] = np.sum(input_layer_neuron[0]*self.weight_layers[i].T,axis=1)
#     elif i==weight_layer_count-1:
#         output_layer_neuron = np.sum(hidden_layer_neuron[i-1]*self.weight_layers[i].T,axis=1)
#     else:
#         hidden_layer_neuron[i] = np.sum(hidden_layer_neuron[i-1]*self.weight_layers[i].T,axis=1)

# print("Input")
# print(input_layer_neuron)
# print("Hidden Layer")
# print(hidden_layer_neuron)
# print("Output")
# print(output_layer_neuron)
# print("Weight")
# print(self.weight_layers)
# print("\n")

# print(type(self.weight_layers[0][0][0]))

In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt

# visual = nx.DiGraph()

# for i in range(modelInstance.input_neuron_count):
#     for j in range(len(modelInstance.weight_layers[0][i])):
#     visual.add_edge()

# # visual.add_edge(1, 2, weight=1.5)
# # visual.add_edge(1, 3, weight=2.0)
# # visual.add_edge(2, 4, weight=0.5)
# # visual.add_edge(3, 4, weight=1.0)
# # visual.add_edge(4, 5, weight=2.5)

# pos = nx.nx_agraph.graphviz_layout(visual, prog="dot", args="-Grankdir=LR")
# nx.draw(visual, pos, with_labels=True, node_color="lightblue", edge_color="gray", node_size=2000, font_size=15)

# edge_labels = nx.get_edge_attributes(visual, 'weight')
# nx.draw_networkx_edge_labels(visual, pos, edge_labels=edge_labels, font_color='red')

# plt.show()

In [ ]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.value, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [ ]:
draw_dot(model.final_error)

In [ ]:
modelInstance = FFNN19(
    output_neuron_count=2,
    hidden_layer_count=1,
    input_neuron_count=2,
)

modelInstance.setHiddenLayerNeuronCount([2])
# modelInstance.zeroWeightInitialization()

modelInstance.uniformWeightDistribution(0,100,1)


modelInstance.weight_layers[0][0][1] = Neuron(0.35)
modelInstance.weight_layers[0][0][2] = Neuron(0.35)

modelInstance.weight_layers[0][1][1] = Neuron(0.15)
modelInstance.weight_layers[0][1][2] = Neuron(0.2)

modelInstance.weight_layers[0][2][1] = Neuron(0.25)
modelInstance.weight_layers[0][2][2] = Neuron(0.30)

modelInstance.weight_layers[1][0][1] = Neuron(0.6)
modelInstance.weight_layers[1][0][2] = Neuron(0.6)


modelInstance.weight_layers[1][1][1] = Neuron(0.4)
modelInstance.weight_layers[1][1][2] = Neuron(0.45)

modelInstance.weight_layers[1][2][1] = Neuron(0.5)
modelInstance.weight_layers[1][2][2] = Neuron(0.55)

modelInstance.weight_layers


In [ ]:
modelInstance._feedforward([[0.5,0.1]])

In [ ]:
modelInstance.visualize()